In [1]:
import torch
import torch.optim as optim
import numpy as np

sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

char_set = list(set(sentence))

x_data = []
y_data = []
sequence_length = 10 # window size
learning_rate = 0.1
input_size = len(char_set)
hidden_size = len(char_set)

### data 처리

In [2]:
for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i : i + sequence_length]
    y_str = sentence[i + 1 : i + sequence_length + 1]
    print (i, x_str, "->", y_str)
    x_data.append(list(map(char_set.index, x_str)))
    y_data.append(list(map(char_set.index, y_str)))

x_one_hot = [np.eye(input_size)[x] for x in x_data]

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

### To tensor

In [3]:
# transform as torch tensor variable
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [4]:
print(X.shape)

torch.Size([170, 10, 25])


### RNN + FC

In [5]:
class Net(torch.nn.Module):
    # network 구성
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

    # 계산
    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [6]:
net = Net(input_size, hidden_size, 2)

### loss, optimizer

In [7]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

### training

In [8]:
# start training
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, input_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    results = outputs.argmax(dim=2)
    predict_str = ""
    # make predict sentence
    # 처음 결과는 전부다 사용하고, 이후 결과는 이전것과 마지막 하나를 빼고 모두 겹침
    for j, result in enumerate(results):
        if j == 0:
            predict_str += ''.join([char_set[t] for t in result])
        else:
            predict_str += char_set[result[-1]]
    print(predict_str)

mmtttlllttltltltttttlltltttllttllllttltfltltlttllfltlllttllltlttllllllltttlltllltlllllttttllltttlltlttlttlllttlllttllltlttllltltttltttlltlttlllltlllttlttltlllttttllltltttttlttlltt
trfuuuuuuuuuuuruuuuurusuuuuuuuuuuuuuuuuuuuuuuuuuuruuuuuuuuruuruuuuuuuuuuuuuusuruuuuuuuuuuuuruuuuuuuuuuuusuruuuuuuuuuuuuuuuuruuuuuuuuuuuuuruuuuruuuruuuruuuuuuuruuuuuuuuuuuuruuuruuu
tttttttttttttttetttttetetttetetttetettetttttttttttetttttttttttttttttttettttttttttttttetttettettttttttttetttetetttetetettttttttttttttttttttettttttttetttetttttttettttttttttttttttett
  fhslfhsl      l    l   l     ll   l   l           ll   l   ll l       l  ll    l  s     ll       l  lll    l  s      l      l       l  l  l    l   l  ll  ll      l     ll   l  l
 dd sf  nf ndftnff fdffdff fffddttffdddfdffdfddfdfdnffddffdfffffff dffddffffdfdfdftnfdddffdftnf dff dffffdffdftdfdtdffft dfffnfdfffddfftftdffffnffdfffdnffdddffnftdddtddtfff dfffnf
    d ro   d      r d                              d               d          d   r r               

lndou wont to build a ship, don't arum up people together to collect wood and don't assign them tasks and work, but rather toach them to long for themendless immensity of the eea.
lndou wont to build a ship, don't arum up people together to collect wood and don't assign them tosks and work, but rather toach the  to long for the endless immensity of the eea.
lndou wont to build a ship, don't arum up people together to collect wood and won't assign them tosks and work, but rather toach them to long for the endless immensity of the eea.
lneou wont to build a ship, don't arum up people together to collect wood and won't assign them tasks and work, but rather toach them to long for the endless immensity of the eea.
omtou wont to luild a ship, don't arum up people together to collect wood and won't assign them tasks and work, but rather toach them to long for the endless immensity of the sea.
omtou wont to build a ship, don't arum up people together to collect wood and won't assign them task